## 训练模型

wiki-cn-cluster.txt 提供额外的语言学特征

In [1]:
from pyhanlp import *

KBeamArcEagerDependencyParser = JClass('com.hankcs.hanlp.dependency.perceptron.parser.KBeamArcEagerDependencyParser')
CTB_ROOT = "ctb8.0-dep"
CTB_TRAIN = CTB_ROOT + "/train.conll"
CTB_DEV = CTB_ROOT + "/dev.conll" 
CTB_TEST = CTB_ROOT + "/test.conll"
CTB_MODEL = CTB_ROOT + "/ctb.bin"
BROWN_CLUSTER = "wiki-cn-cluster.txt"

In [ ]:
parser = KBeamArcEagerDependencyParser.train(CTB_TRAIN, CTB_DEV, BROWN_CLUSTER, CTB_MODEL)
print(parser.parse("人吃鱼"))
score = parser.evaluate(CTB_TEST)
print("UAS=%.1f LAS=%.1f\n" % (score[0], score[1]))

## 基于依存句法树的意见抽取

In [4]:
from pyhanlp import *

CoNLLSentence = JClass('com.hankcs.hanlp.corpus.dependency.CoNll.CoNLLSentence')
CoNLLWord = JClass('com.hankcs.hanlp.corpus.dependency.CoNll.CoNLLWord')
IDependencyParser = JClass('com.hankcs.hanlp.dependency.IDependencyParser')
KBeamArcEagerDependencyParser = JClass('com.hankcs.hanlp.dependency.perceptron.parser.KBeamArcEagerDependencyParser')


parser = KBeamArcEagerDependencyParser()
tree = parser.parse("电池非常棒，机身不长，长的是待机，但是屏幕分辨率不高。")
print(tree)

1	电池	电池	N	NN	_	3	nsubj	_	_
2	非常	非常	A	AD	_	3	advmod	_	_
3	棒	棒	V	VA	_	0	ROOT	_	_
4	，	，	P	PU	_	3	punct	_	_
5	机身	机身	N	NN	_	7	nsubj	_	_
6	不	不	A	AD	_	7	neg	_	_
7	长	长	V	VA	_	3	conj	_	_
8	，	，	P	PU	_	7	punct	_	_
9	长	长	V	VA	_	11	top	_	_
10	的	的	D	DEC	_	9	cpm	_	_
11	是	是	V	VC	_	7	conj	_	_
12	待机	待机	N	NN	_	11	attr	_	_
13	，	，	P	PU	_	3	punct	_	_
14	但是	但是	A	AD	_	18	advmod	_	_
15	屏幕	屏幕	N	NN	_	16	nn	_	_
16	分辨率	分辨率	N	NN	_	18	nsubj	_	_
17	不	不	A	AD	_	18	neg	_	_
18	高	高	V	VA	_	3	conj	_	_
19	。	。	P	PU	_	3	punct	_	_



![jufashu](./句法树en.jpg)

### 找出词性是NN，且作为某个形容词的名词性主语

In [5]:
for word in tree.iterator():
    if word.POSTAG == "NN" and word.DEPREL == "nsubj":
        print("%s = %s" % (word.LEMMA, word.HEAD.LEMMA))

电池 = 棒
机身 = 长
分辨率 = 高


### 检查形容词是否存在否定修饰的支配词

In [6]:
for word in tree.iterator():
    if word.POSTAG == "NN" and word.DEPREL == "nsubj":
        if tree.findChildren(word.HEAD, "neg").isEmpty():
            print("%s = %s" % (word.LEMMA, word.HEAD.LEMMA))
        else:
            print("%s = 不%s" % (word.LEMMA, word.HEAD.LEMMA))

电池 = 棒
机身 = 不长
分辨率 = 不高


### 提取出“待机”与“长”
“待机”与“长”之间的公共父节点为“是”

In [7]:
for word in tree.iterator():
    if word.POSTAG == "NN":
        if word.DEPREL == "nsubj":  # ①属性

            if tree.findChildren(word.HEAD, "neg").isEmpty():
                print("%s = %s" % (word.LEMMA, word.HEAD.LEMMA))
            else:
                print("%s = 不%s" % (word.LEMMA, word.HEAD.LEMMA))
        elif word.DEPREL == "attr":
            top = tree.findChildren(word.HEAD, "top")  # ②主题

            if not top.isEmpty():
                print("%s = %s" % (word.LEMMA, top.get(0).LEMMA))

电池 = 棒
机身 = 不长
待机 = 长
分辨率 = 不高


## Python接口

In [2]:
from pyhanlp import HanLP
print(HanLP.parseDependency("电池非常棒，机身不长，长的是待机，但是屏幕分辨率不高。"))

1	电池	电池	n	n	_	3	主谓关系	_	_
2	非常	非常	d	d	_	3	状中结构	_	_
3	棒	棒	a	a	_	0	核心关系	_	_
4	，	，	wp	w	_	3	标点符号	_	_
5	机身	机身	n	n	_	6	主谓关系	_	_
6	不长	不长	v	v	_	3	并列关系	_	_
7	，	，	wp	w	_	6	标点符号	_	_
8	长	长	a	a	_	10	主谓关系	_	_
9	的	的	u	u	_	8	右附加关系	_	_
10	是	是	v	v	_	6	并列关系	_	_
11	待机	待机	j	j	_	10	动宾关系	_	_
12	，	，	wp	w	_	10	标点符号	_	_
13	但是	但是	c	c	_	17	状中结构	_	_
14	屏幕	屏幕	n	n	_	15	定中关系	_	_
15	分辨率	分辨率	n	n	_	17	主谓关系	_	_
16	不	不	d	d	_	17	状中结构	_	_
17	高	高	a	a	_	10	并列关系	_	_
18	。	。	wp	w	_	3	标点符号	_	_



![jufashu](./句法树.jpg)